In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import time
from urllib.parse import urlencode
import webbrowser

In [ ]:
CLIENT_ID = '7e5b7f9df7fc403cb0bb2f2daececb2a'
CLIENT_SECRET = '5c40be6905884180883e166c03b4df8c'
REDIRECT_URI = 'http://localhost:8888/callback'

In [ ]:
SCOPE = 'user-read-playback-state user-modify-playback-state user-read-currently-playing'

In [ ]:
params = urlencode({
    'client_id': CLIENT_ID,
    'response_type': 'code',
    'redirect_uri': REDIRECT_URI,
    'scope': SCOPE
})

In [ ]:
auth_url = f"https://accounts.spotify.com/authorize?{params}"
webbrowser.open(auth_url)
print("Abra este link no navegador:", auth_url)

In [ ]:
CODE = "AQBx7NT8gMVZz2pxXS8dD6CkuAcuC6G_mXoVOpNe743YK2cG5IXBST9mJxPMKNtQrajRLOliJwLodgu7eb6j88KxY6Rq8k7X2LkHEYrMapHpTL74Z8MGBL0vQ4e3rSzXzfxyCAGOY0LVKkx6RPQam7S4nRex89vQwqHDEZa85AGVmwtYZYHU6ttFvaYq4HYluwOS9a40qKYkXP1R8K2g23Y8bB8n_6FnXpYIpprdPlGg9gnwsauI2GpiN0fwCYYSFoek-6FFt_XDQS9ojpoRqnk6jf6L"

In [ ]:
import requests
import base64

In [ ]:
auth_header = base64.b64encode(f"{CLIENT_ID}:{CLIENT_SECRET}".encode()).decode()

url = "https://accounts.spotify.com/api/token"
headers = {
    "Authorization": f"Basic {auth_header}",
    "Content-Type": "application/x-www-form-urlencoded"
}

In [ ]:
data = {
    "grant_type": "authorization_code",
    "code": CODE,
    "redirect_uri": REDIRECT_URI
}

In [ ]:
response = requests.post(url, headers=headers, data=data)
if response.status_code == 200:
    tokens = response.json()
    print("Access Token:", tokens['access_token'])
    print("Refresh Token:", tokens['refresh_token'])
else:
    print(f"Erro {response.status_code}: {response.text}")

In [ ]:
access_token = tokens['access_token']

In [ ]:
url_busca = "https://api.spotify.com/v1/search"
headers = {
    "Authorization": f"Bearer {access_token}"
}

In [ ]:
nome_musica = "fã número 1"

params = {
    "q": nome_musica,
    "type": "track",
    "limit": 1
}

In [ ]:
response_busca = requests.get(url_busca, headers=headers, params=params)
resultados = response_busca.json()

In [ ]:

if 'tracks' in resultados and resultados['tracks']['items']:
    track = resultados['tracks']['items'][0]
    track_uri = track['uri']
    print(f"Música encontrada: {track['name']} - URI: {track_uri}")

    url_play = "https://api.spotify.com/v1/me/player/play"
    headers_play = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    data_play = {
        "uris": [track_uri]
    }
    response_play = requests.put(url_play, headers=headers_play, json=data_play)

    if response_play.status_code == 204:
        print("Música iniciada com sucesso!")
    else:
        print(f"Erro ao iniciar música: {response_play.status_code} - {response_play.text}")
else:
    print("Nenhuma música encontrada com esse nome.")

In [ ]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    redirect_uri=REDIRECT_URI,
    scope=SCOPE,
    open_browser=True 
))

In [ ]:
devices = sp.devices()

In [ ]:
if not devices['devices']:
    print("⚠ Nenhum dispositivo ativo. Abra o app do Spotify e toque algo manualmente.")
else:
    device = devices['devices'][0]
    device_id = device['id']
    print(f"🎧 Dispositivo ativo: {device['name']}")

    sp.pause_playback(device_id=device_id)
    print("⏸ Música pausada.")
    time.sleep(2)

    sp.start_playback(device_id=device_id)
    print("▶ Música tocando.")

In [ ]:
from agents.spotify.resources.resource_play_music import play_pause_music


play_pause_music("pause")